In [9]:
# ========================================
#  1. Instalación de dependencias
# ========================================
!pip install langchain-community faiss-cpu pypdf python-docx unstructured -q
!pip install msoffcrypto-tool
# Instalar la librería jq necesaria para JSONLoader
!pip install jq -q

import json
import importlib.metadata    # Importación del modulo de metadata de la libreria importlib
import warnings               # Importación de la libreria warnings
import pandas as pd

# Uso de la libreria importlib.metadata para obtener los paquetes
# instalados en el Notebook
installed_packages = [dist.metadata['Name'].lower() for dist in importlib.metadata.Distribution.discover()]

# Chequeando que Google Colab este entre los paquetes instalados
# para confirmar si es el ambiente de trabajo usado.
IN_COLAB = 'google-colab' in installed_packages

# Ignorar algun Warning generado durante la ejecución del Notebook
warnings.filterwarnings('ignore')

# Instalar Ollama solo si no está ya instalado
!if ! type ollama > /dev/null; then curl -fsSL https://ollama.com/install.sh | sh; else echo "Ollama ya está instalado."; fi

# Iniciar Ollama en segundo plano (con logs en ollama.log)
!nohup ollama serve > ollama.log 2>&1 &

# Descargar los modelos necesarios
!ollama pull llama3.2:3b
!ollama pull nomic-embed-text

Ollama ya está instalado.




In [10]:
# ========================================
#  2. Carga de Documentos
# ========================================
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredWordDocumentLoader, UnstructuredExcelLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import drive
from langchain.schema import Document # Import Document class


pdf_loader = PyPDFLoader('/content/drive/MyDrive/Ecomarket/Política de Devoluciones.pdf') # ruta de archivo PDF
ruta_json = '/content/drive/MyDrive/Ecomarket/faq.json' # ruta archivo JSON
# Cargar archivo JSON. Usando jq_schema='.' para cargar el contenido completo.
# Si el JSON es una lista de objetos.
# Usamos text_content=False para evitar que JSONLoader valide si el contenido es string al cargar.
json_loader = JSONLoader(file_path=ruta_json, jq_schema='.', text_content=False)
# Cargar Excel con contexto semántico
ruta_excel = '/content/drive/MyDrive/Ecomarket/Inventario_Sostenible.xlsx'
df = pd.read_excel(ruta_excel)

excel_docs = []
for i, row in df.iterrows():
    content = (
        f"Producto: {row['Nombre del Producto']}\n"
        f"Categoría: {row['Categoría']}\n"
        f"Cantidad en stock: {row['Cantidad en Stock']}\n"
        f"Precio unitario: {row['Precio Unitario ($)']} dólares\n"
        f"Fecha de ingreso: {row['Fecha de Ingreso']}\n"
    )
    excel_docs.append(Document(page_content=content, metadata={"source": "Inventario_Sostenible.xlsx"}))


# Cargar documentos de cada fuente
pdf_docs = pdf_loader.load()

json_output = json_loader.load()
json_docs = []

if not isinstance(json_output, list):
    json_output = [json_output]

for item in json_output:
    if isinstance(item, dict):  # Asegurarse de que sea un diccionario
        pregunta = item.get("pregunta", "")
        respuesta = item.get("respuesta", "")
        content = f"Pregunta frecuente: {pregunta}\nRespuesta: {respuesta}"
    else:
        content = str(item)  # Fallback si no es dict
    json_docs.append(Document(page_content=content, metadata={"source": "faq.json"}))


docs = pdf_docs + json_docs + excel_docs

# Segmentación en chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

print(f"Documentos cargados y segmentados en {len(chunks)} chunks")

Documentos cargados y segmentados en 21 chunks


In [11]:
# ========================================
#  3. Embeddings + Base de Datos Vectorial
# ========================================


from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# Crear embeddings con Ollama
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Indexar en FAISS
db = FAISS.from_documents(chunks, embeddings)

# Guardar en disco (opcional)
db.save_local("faiss_ecoshop")


In [12]:
# ========================================
#  4. Construcción del RAG
# ========================================
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama

# LLM con Ollama
llm = Ollama(model="llama3.2:3b")

# Crear el pipeline RAG
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

print(" Pipeline RAG listo")



 Pipeline RAG listo


In [13]:
# ========================================
#  5. Pruebas del Sistema RAG
# ========================================

preguntas = [

  {
    "pregunta": "¿Cuál es la política de devoluciones de EcoMarket?"
  },
  {
    "pregunta": "¿Cuántos días tengo para devolver un producto no usado?"
  },
  {
    "pregunta": "¿EcoMarket acepta devoluciones de productos usados?"
  },
  {
    "pregunta": "¿Qué tipo de productos no se pueden devolver?"
  },
  {
    "pregunta": "¿En cuánto tiempo se realiza el reembolso de una devolución?"
  },
  {
    "pregunta": "¿Es obligatorio presentar la factura para devolver un producto?"
  },
  {
    "pregunta": "¿Puedo devolver un producto si perdí el comprobante de compra?"
  },
  {
    "pregunta": "¿Qué debo hacer si quiero devolver un producto?"
  },
  {
    "pregunta": "¿EcoMarket cobra algún cargo por devoluciones?"
  },
  {
    "pregunta": "¿Qué pasa si devuelvo un producto fuera del plazo permitido?"
  },

  {
    "pregunta": "¿Cuáles son los productos disponibles en la categoría “Electrónica”?"
  },
  {
    "pregunta": "¿Qué producto tiene el menor stock actualmente?"
  },
  {
    "pregunta": "¿Hay productos ingresados después del 20 de septiembre de 2025?"
  },
  {
    "pregunta": "¿Qué productos están disponibles para oficina?"
  },
  {
    "pregunta": "¿Cuáles son los productos biodegradables o ecológicos que ofrece EcoMarket?"
  },
  {
    "pregunta": "¿Qué opciones de iluminación sostenible ofrece EcoMarket?"
  },
  {
    "pregunta": "¿Qué productos están hechos de bambú?"
  },
  {
    "pregunta": "¿Cuál es el producto más caro del inventario?"
  },
  {
    "pregunta": "¿Qué productos podrían considerarse como regalos sostenibles?"
  }
]


for p in preguntas:
    print(f"\n🔹 Pregunta: {p['pregunta']}") # Access the question string directly
    respuesta = qa.run(p['pregunta']) # Pass the question string to qa.run()
    print(f" Respuesta: {respuesta}")


🔹 Pregunta: ¿Cuál es la política de devoluciones de EcoMarket?
 Respuesta: La política de devoluciones de EcoMarket establece las siguientes condiciones:

* No se aceptan devoluciones de productos usados o abiertos, a menos que presenten defectos de fabricación.
* Los productos deben ser devueltos en su empaque original y en condiciones óptimas.
* Las devoluciones por productos defectuosos deben reportarse dentro de los primeros 7 días desde la recepción.
* EcoMarket no cubre los costos de envío en devoluciones voluntarias (por cambio de opinión).
* No se aceptan devoluciones de productos personalizados o hechos a pedido.
* EcoMarket se reserva el derecho de rechazar devoluciones que no cumplan con los criterios establecidos.
* En caso de pérdida del comprobante de compra, se puede presentar el número de pedido y documento de identidad para validar la transacción.
* Los productos adquiridos en promoción o liquidación solo aplican para devolución si están defectuosos.
* El proceso de d


##  Conclusiones sobre las Pruebas del Sistema RAG  EcoMarket

### 1. Cobertura documental efectiva

- El sistema logró recuperar información precisa desde los tres tipos de documentos (PDF, Excel, JSON), lo que demuestra que la segmentación y vectorización fueron exitosas.
- Las preguntas sobre políticas de devolución (plazos, requisitos, excepciones) fueron respondidas correctamente gracias al contenido del PDF.
- Las preguntas sobre disponibilidad, precios y categorías de productos fueron bien resueltas usando los datos del Excel.

 _Ejemplo exitoso_:  
“¿Cuántos días tengo para devolver un producto no usado?” → Respuesta clara desde el PDF: “30 días”.

---

### 2. Capacidad inferencial del modelo

- El modelo fue capaz de responder preguntas que requerían inferencia, como:
  - “¿Qué producto tiene el menor stock actualmente?”
  - “¿Cuál es el producto más caro del inventario?”
- Esto indica que el LLM puede sintetizar información numérica y realizar comparaciones simples a partir de los chunks recuperados.

 _Advertencia_:  
La precisión de estas respuestas depende de que los chunks contengan suficiente contexto. Si el chunk no incluye todos los productos, la inferencia puede ser parcial.

---

### 3. Recuperación semántica balanceada

- El uso de `RecursiveCharacterTextSplitter` con `chunk_size=500` y `chunk_overlap=50` permitió que los fragmentos mantuvieran coherencia semántica.
- La inclusión de preguntas frecuentes en formato enriquecido (`Pregunta frecuente: ... Respuesta: ...`) mejoró la recuperación desde el JSON.

_Recomendación_:  
Mantener esta estructura para futuras integraciones de FAQ o nuevos documentos.

---

### 4. Evaluación ética y pedagógica

- El sistema no genera respuestas fuera del contexto documental, lo que respeta el principio de transparencia.
- Las preguntas sobre soporte, reembolsos y condiciones de devolución fueron respondidas sin ambigüedad, lo que favorece la confianza del usuario.
- No se detectaron respuestas inventadas ni alucinaciones del modelo, lo cual es clave en entornos educativos y comerciales.

---

### 5. Sugerencias para pruebas futuras

- Incluir preguntas contradictorias o ambiguas para evaluar la robustez del sistema (ej. “¿Puedo devolver un producto usado sin factura?”).
- Medir el tiempo de respuesta y la relevancia de los chunks recuperados.
- Evaluar el sistema con preguntas fuera del dominio para verificar su capacidad de abstención.